
#  JANUS: A bit-wise reversible integrator for N-body simulations

Hanno Rein<br/>
Daniel Tamayo 

# Time reversibility
* Hamiltonian systems are time reversible.
* Yet all integrators used for planetary dynamics are not time reversible!

In [1]:
import rebound
from numpy import random
from scipy import misc

def fromFile(f):
    ji = misc.imread(f)[:,:,0]
    sim = rebound.Simulation()
    while sim.N<1000:
        y, x = random.randint(ji.shape[0]),random.randint(ji.shape[1])
        if ji[y,x]==0:
            sim.add(m=1,x=x,y=-y,z=10*random.normal())
    sim.move_to_com()
    sim.dt = 1
    sim.softening = 2
    return sim

In [2]:
sim = fromFile("lf.png")
sim.integrator="leapfrog"
sim.getWidget(size=(500,300),orbits=False,scale=200)

In [3]:
sim.integrate(500)

In [4]:
sim.dt *=-1
sim.integrate(0)

## Why is leap frog not time reversible?
* Floating point numbers!
* $a + b = c\; $  does not imply $\;  c - b = a$
* $a + b = c\; $  and $\; a + d = c \;$ does not imply $\; b = d$
* Information is lost, entropy generated
* At every calculation!

In [5]:
1.25 + 7.79 == 9.04

True

In [6]:
1.25 == 9.04 - 7.79

False

## Liouville's Theorem
* Phase space density is conserved along trajectories
* Not the case with a non-reversible integrator
<img src="states.png" height="50%" width="50%">

## How Janus works
* Integer arithmetic!
* $a + b = c\; $  implies $\;  c - b = a$
* $a + b = c\; $  and $\; a + d = c \;$ implies $\; b = d$
* Positions and velocities are stored on integer grid
* Floating point arithmetic is used to calculate acceleration

In [7]:
125 + 779 == 904

True

In [8]:
125 == 904 - 779

True

## Drift Step

<img src="drift.png" height="50%" width="50%">

## Kick Step

<img src="kick.png"  height="50%" width="50%">

In [9]:
sim = fromFile("janus.png")
sim.integrator="janus"
sim.ri_janus.order=2
sim.getWidget(size=(500,300),orbits=False,scale=200)

In [10]:
sim.integrate(500)

In [11]:
sim.dt *=-1
sim.integrate(0)

In [12]:
sim = rebound.Simulation()
sim.add(m=1)
for i in range(10):
    sim.add(m=1e-3, a=1+0.1*i,l=i, inc=0.01*i)
sim.move_to_com()
sim.integrator = "whfast"
sim.getWidget(scale=3, size=(500,200))

In [13]:
sim.dt = 0.01  # forward
sim.integrate(200,exact_finish_time=0)

In [14]:
sim.dt = -0.01 # backward
sim.integrate(0,exact_finish_time=0)

In [15]:
sim = rebound.Simulation()
sim.add(m=1)
for i in range(10): sim.add(m=1e-3, a=1+0.1*i,l=i, inc=0.01*i)
sim.move_to_com()
sim.integrator = "janus"
sim.getWidget(scale=3, size=(500,200))

In [16]:
sim.dt = 0.01  # forward
sim.integrate(200,exact_finish_time=0)

In [17]:
sim.dt = -0.01 # backward
sim.integrate(0,exact_finish_time=0)

# Summary
<img src="janus.jpg">

* JANUS, truly bit-wise reversible integrator
* Symplectic
* Order 2, 4, 6, 8 or 10
* Fast and accurate enough to integrate planetary systems over astronomical timescales
* Integrated in REBOUND
* arxiv:1704.07715

# Does it matter?
* Philosophically: yes. 
* In practice: I don't know!
* Preliminary results of integrations of the Solar System over 200 Myr show same statistical evolution
<img src="comp.png">


